In [1]:
import requests
import pandas as pd
import numpy as np
# 1. API endpoint
url = "https://api.census.gov/data/2023/pep/charv"

# 2. Parameters
params = {
    "get": "NAME,STATE,YEAR,POP,AGE,SEX",
    "for": "state:*"
}

# 3. Send request
response = requests.get(url, params=params)

# 4. Parse JSON
data = response.json()

# 5. Convert to DataFrame
pop_df = pd.DataFrame(data[1:], columns=data[0])

pop_df.sample(5)


,NAME,STATE,YEAR,POP,AGE,SEX,state
25874,Illinois,17,2023,82089,2600,1,17
52945,New Jersey,34,2020,126805,5100,0,34
68960,Pennsylvania,42,2020,564190,1824,2,42
26388,Illinois,17,2022,74112,6500,1,17
27113,Indiana,18,2022,48767,2100,2,18


In [2]:
pop_df.to_csv(
    "population_dataset.csv",
    index=False
)

In [3]:
pop_df.sample(5)

,NAME,STATE,YEAR,POP,AGE,SEX,state
85187,Puerto Rico,72,2021,19815,1900,2,72
27009,Illinois,17,2022,84292,2800,2,17
48401,Montana,30,2020,36326,5559,2,30
86928,Wisconsin,55,2022,57808,7200,0,55
35593,Maine,23,2022,16749,4200,0,23


In [4]:
print(pop_df.shape)
print(type(pop_df.AGE[0]))

(90480, 7)
<class 'str'>


In [5]:
def fetch_acs_median_income_state(year):
    """
    grab ACS 1-year meadian household income in past 12 months for all states.
    endpoint: https://api.census.gov/data/{year}/acs/acs1
    variable: B19013_001E
    """
    url = f"https://api.census.gov/data/{year}/acs/acs1"
    params = {
        "get": "NAME,B19013_001E",
        "for": "state:*",
    }

    r = requests.get(url, params=params, timeout=60)

    # raise error if not 200
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {r.text[:300]}")

    data = r.json()
    df = pd.DataFrame(data[1:], columns=data[0])

    # clean
    df["YEAR"] = year
    df = df.rename(columns={"state": "STATE", "B19013_001E": "median_hh_income"})
    df["STATE"] = df["STATE"].astype(str).str.zfill(2)
    df["median_hh_income"] = pd.to_numeric(df["median_hh_income"], errors="coerce")

    return df[["STATE", "YEAR", "NAME", "median_hh_income"]]


years = [2021, 2022, 2023]
acs_income = pd.concat([fetch_acs_median_income_state(y) for y in years], ignore_index=True)

acs_income.head()


,STATE,YEAR,NAME,median_hh_income
0,01,2021,Alabama,53913
1,72,2021,Puerto Rico,22237
2,04,2021,Arizona,69056
3,05,2021,Arkansas,52528
4,06,2021,California,84907


In [6]:
acs_income.to_csv(
    "median_income_state.csv",
    index=False
)

In [7]:
print(len(acs_income.STATE.unique()))
print(acs_income.STATE.unique())


52
<StringArray>
['01', '72', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
 '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
 '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42',
 '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '02']
Length: 52, dtype: str


In [8]:
pop_df["STATE"] = pop_df["STATE"].astype(str).str.zfill(2)
acs_income["STATE"] = acs_income["STATE"].astype(str).str.zfill(2)

pop_df["YEAR"] = pop_df["YEAR"].astype(int)
acs_income["YEAR"] = acs_income["YEAR"].astype(int)

merged_df = pop_df.merge(
    acs_income[["STATE", "YEAR", "median_hh_income"]],
    on=["STATE", "YEAR"],
    how="left"
)

merged_df.to_csv(
    "merged_dataset.csv",
    index=False
)

In [9]:
merged_df.sample(5)

,NAME,STATE,YEAR,POP,AGE,SEX,state,median_hh_income
18642,Florida,12,2020,143421,3100,1,12,NaN
59565,North Dakota,38,2022,9736,1700,0,38,71970.0
444,Alabama,01,2023,66322,6300,0,01,62212.0
10131,California,06,2020,273092,3600,2,06,NaN
51475,Nebraska,31,2020,11896,3100,2,31,NaN


Data Cleaning

In [10]:
print(merged_df.shape)

print(merged_df.columns)

print(merged_df.describe(include='all'))

(90480, 8)
Index(['NAME', 'STATE', 'YEAR', 'POP', 'AGE', 'SEX', 'state',
       'median_hh_income'],
      dtype='str')
           NAME  STATE          YEAR    POP    AGE    SEX  state  \
count     90480  90480  90480.000000  90480  90480  90480  90480   
unique       52     52           NaN  67557    116      3     52   
top     Alabama     01           NaN   6037   0000      0     01   
freq       1740   1740           NaN      9    780  30160   1740   
mean        NaN    NaN   2021.200000    NaN    NaN    NaN    NaN   
std         NaN    NaN      1.166197    NaN    NaN    NaN    NaN   
min         NaN    NaN   2020.000000    NaN    NaN    NaN    NaN   
25%         NaN    NaN   2020.000000    NaN    NaN    NaN    NaN   
50%         NaN    NaN   2021.000000    NaN    NaN    NaN    NaN   
75%         NaN    NaN   2022.000000    NaN    NaN    NaN    NaN   
max         NaN    NaN   2023.000000    NaN    NaN    NaN    NaN   

        median_hh_income  
count       54288.000000  
unique   

In [11]:
merged_df = merged_df.rename(columns=({
    'NAME': 'State_name',
    'STATE': 'State_code',
    'YEAR': 'Year',
    'POP': 'Population',
    'AGE': 'Age',
    'SEX': 'Sex',
    'median_hh_income': 'Median_household_income'
}
))

# Drop duplicate 'state' column
if 'state' in merged_df.columns:
    merged_df = merged_df.drop(columns=['state'])

merged_df.head(5)

,State_name,State_code,Year,Population,Age,Sex,Median_household_income
0,Alabama,01,2020,5024294,0000,0,NaN
1,Alabama,01,2020,5031864,0000,0,NaN
2,Alabama,01,2021,5050380,0000,0,53913.0
3,Alabama,01,2022,5073903,0000,0,59674.0
4,Alabama,01,2023,5108468,0000,0,62212.0


In [12]:
# Alter data type
merged_df['Year'] = merged_df['Year'].astype(int)
merged_df['Population'] = pd.to_numeric(merged_df['Population'], errors='coerce')
merged_df['State_code'] = merged_df['State_code'].astype(str).str.zfill(2)
merged_df['Median_household_income'] = pd.to_numeric(merged_df['Median_household_income'], 
                                                     errors='coerce')
for col in ['State_name', 'Sex']:
    merged_df[col] = merged_df[col].astype('category')

# Drop rows with missing values in 'Median_household_income' and filter for years 2021 and later
# Since the income data is only for 2021-2023, we can safely filter for those years without losing any data.
merged_df_clean = merged_df[merged_df["Year"] >= 2021].copy()

In [13]:
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Age,Sex,Median_household_income
2,Alabama,01,2021,5050380,0000,0,53913.0
3,Alabama,01,2022,5073903,0000,0,59674.0
4,Alabama,01,2023,5108468,0000,0,62212.0
7,Alabama,01,2021,56741,0001,0,53913.0
8,Alabama,01,2022,57666,0001,0,59674.0


In [14]:
for col in merged_df_clean.columns:
    print(f"{col}: {merged_df_clean[col].dtype}")

State_name: category
State_code: str
Year: int64
Population: int64
Age: str
Sex: category
Median_household_income: float64


In [15]:
merged_df_clean.Age.unique()

<StringArray>
['0000', '0001', '0100', '0200', '0300', '0400', '0401', '0499', '0500',
 '0509',
 ...
 '7700', '7800', '7900', '8000', '8084', '8100', '8200', '8300', '8400',
 '8599']
Length: 116, dtype: str

In [16]:

merged_df_clean["Age"] = merged_df_clean["Age"].astype(str)
def is_valid_age_code(age_code):
    # keep all ages population
    if age_code == "0000":
        return True
    
    # keep 85+ group
    if age_code == "8599":
        return True
    
    # find single age groups that end with "00" (e.g., "0100", "0200", ..., "8400")
    if age_code.endswith("00") and age_code != "0000":
        return True
    
    return False
merged_df_clean = merged_df_clean[merged_df_clean["Age"].apply(is_valid_age_code)].copy()
merged_df_clean["Age"].unique()

<StringArray>
['0000', '0100', '0200', '0300', '0400', '0500', '0600', '0700', '0800',
 '0900', '1000', '1100', '1200', '1300', '1400', '1500', '1600', '1700',
 '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600',
 '2700', '2800', '2900', '3000', '3100', '3200', '3300', '3400', '3500',
 '3600', '3700', '3800', '3900', '4000', '4100', '4200', '4300', '4400',
 '4500', '4600', '4700', '4800', '4900', '5000', '5100', '5200', '5300',
 '5400', '5500', '5600', '5700', '5800', '5900', '6000', '6100', '6200',
 '6300', '6400', '6500', '6600', '6700', '6800', '6900', '7000', '7100',
 '7200', '7300', '7400', '7500', '7600', '7700', '7800', '7900', '8000',
 '8100', '8200', '8300', '8400', '8599']
Length: 86, dtype: str

In [17]:
mask = merged_df_clean["Age"].eq("0000") | merged_df_clean["Age"].eq("8599") | merged_df_clean["Age"].str.match(r"^\d{2}00$")
merged_df_clean = merged_df_clean[mask].copy()

def age_code_to_year(code: str):
    if code == "0000":
        return np.nan          
    if code == "8599":
        return 85              # use 85 represent 85+
    return int(code[:2])

merged_df_clean["Age_year"] = merged_df_clean["Age"].apply(age_code_to_year)

# create a new column "Age_label" for better readability
merged_df_clean["Age_label"] = np.where(merged_df_clean["Age"].eq("0000"), "All ages",
                   np.where(merged_df_clean["Age"].eq("8599"), "85+",
                            merged_df_clean["Age_year"].astype("Int64").astype(str)))

merged_df_clean.Age_label.unique()

<StringArray>
['All ages',        '1',        '2',        '3',        '4',        '5',
        '6',        '7',        '8',        '9',       '10',       '11',
       '12',       '13',       '14',       '15',       '16',       '17',
       '18',       '19',       '20',       '21',       '22',       '23',
       '24',       '25',       '26',       '27',       '28',       '29',
       '30',       '31',       '32',       '33',       '34',       '35',
       '36',       '37',       '38',       '39',       '40',       '41',
       '42',       '43',       '44',       '45',       '46',       '47',
       '48',       '49',       '50',       '51',       '52',       '53',
       '54',       '55',       '56',       '57',       '58',       '59',
       '60',       '61',       '62',       '63',       '64',       '65',
       '66',       '67',       '68',       '69',       '70',       '71',
       '72',       '73',       '74',       '75',       '76',       '77',
       '78',       '79',       '80', 

In [18]:
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Age,Sex,Median_household_income,Age_year,Age_label
2,Alabama,01,2021,5050380,0000,0,53913.0,NaN,All ages
3,Alabama,01,2022,5073903,0000,0,59674.0,NaN,All ages
4,Alabama,01,2023,5108468,0000,0,62212.0,NaN,All ages
12,Alabama,01,2021,57503,0100,0,53913.0,1.0,1
13,Alabama,01,2022,57244,0100,0,59674.0,1.0,1


In [19]:
merged_df_clean.drop(columns=["Age"], inplace=True)

In [20]:
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Sex,Median_household_income,Age_year,Age_label
2,Alabama,01,2021,5050380,0,53913.0,NaN,All ages
3,Alabama,01,2022,5073903,0,59674.0,NaN,All ages
4,Alabama,01,2023,5108468,0,62212.0,NaN,All ages
12,Alabama,01,2021,57503,0,53913.0,1.0,1
13,Alabama,01,2022,57244,0,59674.0,1.0,1


In [21]:
for col in merged_df_clean.columns:
    print(f"{col}: {merged_df_clean[col].dtype}")

State_name: category
State_code: str
Year: int64
Population: int64
Sex: category
Median_household_income: float64
Age_year: float64
Age_label: str


In [22]:
# 1) create Age_group column
merged_df_clean["Age_group"] = pd.NA

# 2) keep All ages / 85+ as separate groups
is_all = merged_df_clean["Age_label"].astype(str).str.strip().str.lower().eq("all ages")
is_85p = merged_df_clean["Age_label"].astype(str).str.strip().str.lower().eq("85+")
merged_df_clean.loc[is_all, "Age_group"] = "All ages"
merged_df_clean.loc[is_85p, "Age_group"] = "85+"

# 3) Create age groups: 0-18, 19-30, 31-45, 46-60, 61-75, 76-84, 85+
bins = [-0.1, 18, 30, 45, 60, 75, 84, 200]
labels = ["0-18", "19-30", "31-45", "46-60", "61-75", "76-84", "85+"]

mask_other = (~is_all) & (~is_85p) & (merged_df_clean["Age_year"].notna())

merged_df_clean.loc[mask_other, "Age_group"] = pd.cut(
    merged_df_clean.loc[mask_other, "Age_year"].astype(float),
    bins=bins,
    labels=labels
).astype(str)

# 4) check results
merged_df_clean["Age_group"].value_counts(dropna=False)


Age_group
0-18        8424
31-45       7020
46-60       7020
61-75       7020
19-30       5616
76-84       4212
All ages     468
85+          468
Name: count, dtype: int64

In [23]:
merged_df_clean.drop(columns=["Age_year"], inplace=True)

In [24]:
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group
2,Alabama,01,2021,5050380,0,53913.0,All ages,All ages
3,Alabama,01,2022,5073903,0,59674.0,All ages,All ages
4,Alabama,01,2023,5108468,0,62212.0,All ages,All ages
12,Alabama,01,2021,57503,0,53913.0,1,0-18
13,Alabama,01,2022,57244,0,59674.0,1,0-18


In [25]:
# Missing value check
print(merged_df_clean.isna().sum())
merged_df_clean = merged_df_clean.reset_index(drop=True)
merged_df_clean.head(5)

State_name                 0
State_code                 0
Year                       0
Population                 0
Sex                        0
Median_household_income    0
Age_label                  0
Age_group                  0
dtype: int64


,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group
0,Alabama,01,2021,5050380,0,53913.0,All ages,All ages
1,Alabama,01,2022,5073903,0,59674.0,All ages,All ages
2,Alabama,01,2023,5108468,0,62212.0,All ages,All ages
3,Alabama,01,2021,57503,0,53913.0,1,0-18
4,Alabama,01,2022,57244,0,59674.0,1,0-18


In [26]:
# Remove duplicate rows

merged_df_clean = merged_df_clean.drop_duplicates()

print(merged_df_clean.duplicated().sum())

0


In [27]:
merged_df_clean.groupby(['State_code', 'Year'])['Median_household_income'].nunique().max()

np.int64(1)

The median household income variable is measured at the state–year level.
It represents the overall median household income of each state in a given year and is merged onto age- and sex-specific population records for analytical purposes. Therefore, income values are constant across age and sex groups within the same state and year.

In [28]:
merged_df_clean.to_csv('Cleaned_merged_dataset.csv', index=False)

In [30]:
# Relative income
merged_df_clean['state_income_mean'] = merged_df_clean.groupby('State_name')['Median_household_income'].transform('mean')
merged_df_clean['state_income_std'] = merged_df_clean.groupby('State_name')['Median_household_income'].transform('std')

merged_df_clean['income_relative'] = (merged_df_clean['Median_household_income'] - merged_df_clean['state_income_mean']) / merged_df_clean['state_income_std']
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group,state_income_mean,state_income_std,income_relative
0,Alabama,01,2021,5050380,0,53913.0,All ages,All ages,58599.666667,3474.419749,-1.348906
1,Alabama,01,2022,5073903,0,59674.0,All ages,All ages,58599.666667,3474.419749,0.309212
2,Alabama,01,2023,5108468,0,62212.0,All ages,All ages,58599.666667,3474.419749,1.039694
3,Alabama,01,2021,57503,0,53913.0,1,0-18,58599.666667,3474.419749,-1.348906
4,Alabama,01,2022,57244,0,59674.0,1,0-18,58599.666667,3474.419749,0.309212


The relative income measures deviation from long-term state income level.

In [ ]:
# Demographic decomposition
merged_df_clean['state_year_total_pop'] = merged_df_clean.groupby(['State_name', 'Year'])['Population'].transform('sum')

merged_df_clean['population_share'] = merged_df_clean['Population'] / merged_df_clean['state_year_total_pop']
merged_df_clean.head(5)

,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group,state_income_mean,state_income_std,income_relative,state_year_total_pop,population_share
0,Alabama,01,2021,5050380,0,53913.0,All ages,All ages,58599.666667,3474.419749,-1.348906,20088038,0.251412
1,Alabama,01,2022,5073903,0,59674.0,All ages,All ages,58599.666667,3474.419749,0.309212,20180280,0.251429
2,Alabama,01,2023,5108468,0,62212.0,All ages,All ages,58599.666667,3474.419749,1.039694,20318102,0.251424
3,Alabama,01,2021,57503,0,53913.0,1,0-18,58599.666667,3474.419749,-1.348906,20088038,0.002863
4,Alabama,01,2022,57244,0,59674.0,1,0-18,58599.666667,3474.419749,0.309212,20180280,0.002837


The demographic decomposition captures demographic structure within each state-year.

In [34]:
# Stability indicator
merged_df_clean["population_volatility"] = (
    merged_df_clean.groupby("State_name")["Population"]
    .transform("std")
)

merged_df_clean.head(5)


,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group,state_income_mean,state_income_std,income_relative,state_year_total_pop,population_share,population_volatility
0,Alabama,01,2021,5050380,0,53913.0,All ages,All ages,58599.666667,3474.419749,-1.348906,20088038,0.251412,381851.938371
1,Alabama,01,2022,5073903,0,59674.0,All ages,All ages,58599.666667,3474.419749,0.309212,20180280,0.251429,381851.938371
2,Alabama,01,2023,5108468,0,62212.0,All ages,All ages,58599.666667,3474.419749,1.039694,20318102,0.251424,381851.938371
3,Alabama,01,2021,57503,0,53913.0,1,0-18,58599.666667,3474.419749,-1.348906,20088038,0.002863,381851.938371
4,Alabama,01,2022,57244,0,59674.0,1,0-18,58599.666667,3474.419749,0.309212,20180280,0.002837,381851.938371


The population volatility measures demographic stability within each state.

In [36]:
# Scale interaction
merged_df_clean["income_per_population"] = merged_df_clean["Median_household_income"] / merged_df_clean["Population"]

merged_df_clean.head(5)


,State_name,State_code,Year,Population,Sex,Median_household_income,Age_label,Age_group,state_income_mean,state_income_std,income_relative,state_year_total_pop,population_share,population_volatility,income_per_population
0,Alabama,01,2021,5050380,0,53913.0,All ages,All ages,58599.666667,3474.419749,-1.348906,20088038,0.251412,381851.938371,0.010675
1,Alabama,01,2022,5073903,0,59674.0,All ages,All ages,58599.666667,3474.419749,0.309212,20180280,0.251429,381851.938371,0.011761
2,Alabama,01,2023,5108468,0,62212.0,All ages,All ages,58599.666667,3474.419749,1.039694,20318102,0.251424,381851.938371,0.012178
3,Alabama,01,2021,57503,0,53913.0,1,0-18,58599.666667,3474.419749,-1.348906,20088038,0.002863,381851.938371,0.937568
4,Alabama,01,2022,57244,0,59674.0,1,0-18,58599.666667,3474.419749,0.309212,20180280,0.002837,381851.938371,1.042450


This captures income scale relative to population size.